In [1]:
import cv2
import numpy as np

# Main Code and Logic

### reading video


In [2]:
cap=cv2.VideoCapture('video.mp4')

### Initialize Subtructor

In [3]:
algo=cv2.bgsegm.createBackgroundSubtractorMOG()

### Line initialize

In [4]:
Line_position= 550

### initialize min_width and min_height of rectangle


In [5]:
mini_width=80
mini_height=80

### to count

In [6]:
def center_handle(x,y,w,h):
    dx=int(w/2)
    dy=int(h/2)
    cx=x+dx
    cy=y+dy
    
    return cx,cy

In [7]:
#initialize list of detected vehicles
detect=[]

#allowable error between pixel
offset=10

# initialize cnt for vehicle count
cnt=0

### XML for cars


In [8]:
# car_cascade_src = 'cars.xml'
# car_cascade = cv2.CascadeClassifier(car_cascade_src)

In [9]:
while True:
    
    # cap.read() return two values 1-> bool type 2-> frame
    ret,frame=cap.read()
    # when video is finished i.e ret==0
    if ret==False:
        break
    
    
    
    
    
    #convert the image from one color to another
    grey=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    # making image black and white background-> black, vehicles->white
    blur=cv2.GaussianBlur(grey,(3,3),5)
    #apply to all incoming frame 1 by 1
    img_sub=algo.apply(blur)
    dilat=cv2.dilate(img_sub,np.ones((5,5)))
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilatada=cv2.morphologyEx(dilat,cv2.MORPH_CLOSE,kernel)
    dilatada=cv2.morphologyEx(dilatada,cv2.MORPH_CLOSE,kernel)
    counterShape, h=cv2.findContours(dilatada,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        
    
    
    # draw line
    cv2.line(frame,(25,Line_position),(1200,Line_position),(255,255,255),1)
    
    
    
    
    # draw rectangle for vehicles
    #passing tuple c->channel
    for (i,c) in enumerate(counterShape):
        (x,y,w,h)=cv2.boundingRect(c)
        validate_counter=(w>=mini_width) and (h>=mini_height)
        if not validate_counter:
            continue
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.putText(frame,str(cnt),(x+15,y-15),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255),1)
    
        
        
        # points to count vehicle
        center=center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(frame,center,1,(0,0,255),-1)
        
          # if vehicle cross the line then count
        for (x,y) in detect:
            if y<(Line_position+offset) and y>(Line_position-offset):
                cnt+=1
            cv2.line(frame,(25,Line_position),(1200,Line_position),(255,255,255),2)
            detect.remove((x,y))
            
    cv2.putText(frame,'Counter:'+str(cnt),(25,35),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255),2)
    
    
    
    
    
    
    
    
    
    
    
    
    cv2.imshow('main video',frame)
    
    # stoping condition of vdo, cv2.waitKey(1)-> will wait for 1 millisecond when 13 ASCII for 'ENTER' key is pressed
    
    if cv2.waitKey(1)==13:
        break
# to destroy all windows
cv2.destroyAllWindows()
cap.release()